In [1]:
import re
import aocd
from dataclasses import dataclass
from typing import Self
import networkx as nx

In [2]:
@dataclass
class Part:
    x: int
    m: int
    a: int
    s: int
    
    @classmethod
    def parse(cls, string) -> Self:
        x, m, a, s = re.findall(r'\d+', string)
        return cls(int(x), int(m), int(a), int(s))

@dataclass
class Rule:
    condition: str | None
    workflow: str

    def check(self, x: int, m: int, a: int, s: int) -> bool:
        if not self.condition:
            return True
        return eval(self.condition)

    @classmethod
    def parse(cls, string) -> Self:
        match string.split(':'):
            case condition, workflow:
                return cls(condition, workflow)
            case [workflow]:
                return cls(None, workflow)
        raise Exception("could not parse rule")
        

@dataclass
class Workflow:
    name: str
    rules: list[Rule]

    def consider(self, part: Part, workflows: dict[str, Self]) -> str:
        for rule in self.rules:
            if rule.check(part.x, part.m, part.a, part.s):
                if rule.workflow in workflows.keys():
                    return workflows[rule.workflow].consider(part, workflows)
                return rule.workflow
        raise Exception("no workflow")

    @classmethod
    def parse(cls, string) -> Self:
        if m := re.match(r'(\w+){(.+)}', string):
            name = m[1]
            rules = [Rule.parse(s) for s in m[2].split(',')]
            return cls(name, rules)
        raise Exception("could not parse workflow")


In [3]:
data = aocd.get_data(day=19, year=2023)
workflow_lines, part_lines = [chunk.splitlines() for chunk in data.split('\n\n')]
parts = [Part.parse(line) for line in part_lines]
workflows = {w.name: w for w in (Workflow.parse(line) for line in workflow_lines)}

In [4]:
s = 0
for part in parts:
    if workflows['in'].consider(part, workflows) == 'A':
        s += part.x + part.m + part.a + part.s

print("Part 1:", s)

Part 1: 376008


In [5]:
G = nx.DiGraph()
for workflow in workflows.values():
    for num, rule in enumerate(workflow.rules):
        rule_name = f"{workflow.name}{"'" * num}"
        if rule.condition:
            G.add_edge(rule_name, rule.workflow, condition=rule.condition)
            G.add_edge(rule_name, rule_name+"'", condition="not "+rule.condition)
        else:
            G.add_edge(rule_name, rule.workflow, condition='')

In [6]:
num_combinations = 0

for path in nx.all_simple_edge_paths(G, 'in', 'A'):
    x_s = list(range(1, 4001))
    m_s = list(range(1, 4001))
    a_s = list(range(1, 4001))
    s_s = list(range(1, 4001))

    for edge in path:
        condition = G.edges[edge]['condition']
        x_s = [x for x in x_s if eval(condition)] if 'x' in condition else x_s
        m_s = [m for m in m_s if eval(condition)] if 'm' in condition else m_s
        a_s = [a for a in a_s if eval(condition)] if 'a' in condition else a_s
        s_s = [s for s in s_s if eval(condition)] if 's' in condition else s_s
    
    num_combinations += len(x_s) * len(m_s) * len(a_s) * len(s_s)

print("Part 2:", num_combinations)

Part 2: 124078207789312
